# static_rnn vs. dynamic_rnn

You may have noticed that Tensorflow contains two different functions for RNNs: [tf.nn.static_rnn](https://www.tensorflow.org/api_docs/python/tf/nn/static_rnn) and [tf.nn.dynamic_rnn](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn). Which one to use?


Internally, tf.nn.rnn creates an unrolled graph for a fixed RNN length. That means, if you call tf.nn.rnn with inputs having 200 time steps you are creating a static graph with 200 RNN steps. First, graph creation is slow. Second, you’re unable to pass in longer sequences (> 200) than you’ve originally specified.

`tf.nn.dynamic_rnn` solves this. It uses a `tf.While` loop to dynamically construct the graph when it is executed. That means graph creation is faster and you can feed batches of variable size. What about performance? You may think the static rnn is faster than its dynamic counterpart because it pre-builds the graph. In my experience that’s not the case.

In short, just use `tf.nn.dynamic_rnn`. 

In [32]:
import tensorflow as tf
import numpy as np

In [33]:
tf.reset_default_graph()
sess = tf.Session()

BATCH_SIZE = 2
SEQUENCE_LENGTH = 10
NUMBER_OF_WORDS = 8
# Create input data
X = np.random.randn(BATCH_SIZE, SEQUENCE_LENGTH, NUMBER_OF_WORDS)

# The second example is of length 6 
X[1,6:] = 0
X_lengths = [10, 6]




In [34]:
with tf.device(":/cpu:0"):
    data = tf.placeholder(tf.float64, [None,  SEQUENCE_LENGTH, NUMBER_OF_WORDS])
    cell = tf.nn.rnn_cell.LSTMCell(num_units=8, state_is_tuple=True)

    # defining initial state
    initial_state = cell.zero_state(BATCH_SIZE, dtype=tf.float64)

    encode = tf.nn.dynamic_rnn(
        cell=cell,
        dtype=tf.float64,
        sequence_length=X_lengths,
        inputs=data,
        initial_state=initial_state)

In [38]:
sess.run(tf.global_variables_initializer())
outputs, last_states = sess.run(encode, feed_dict={data: X})

In [39]:
outputs

array([[[-0.0283708 , -0.02738941,  0.0755726 , -0.08008824,  0.01081016,
         -0.03213166, -0.0536146 , -0.15033308],
        [-0.07012742,  0.07191048,  0.16121555, -0.06183664,  0.10730578,
          0.05112168,  0.20581951, -0.03912028],
        [ 0.1738402 ,  0.26620728,  0.10301632,  0.13077066,  0.0349601 ,
          0.0586684 ,  0.27258061,  0.32184957],
        [-0.00453193,  0.05837649,  0.00355916, -0.08189783,  0.01207369,
          0.08217721,  0.09261072,  0.1040738 ],
        [ 0.07379494,  0.06887853,  0.02938879, -0.04013838, -0.01955719,
          0.02584432,  0.09734795,  0.12879024],
        [ 0.20445797,  0.04512449, -0.03814287,  0.10622528,  0.02263981,
         -0.17649335,  0.24971871,  0.36151143],
        [ 0.20608251,  0.13480439,  0.10275332,  0.04439773, -0.00303557,
         -0.27451886,  0.17191344,  0.31303354],
        [ 0.32365047,  0.48470812,  0.17407803,  0.2052482 , -0.02023543,
         -0.17407063,  0.29517553,  0.48837349],
        [ 0.1345

In [40]:
last_states

LSTMStateTuple(c=array([[ 0.26831959,  0.33156717,  0.33671164,  0.2987655 , -0.24417013,
        -0.92449115,  0.30798403,  0.61355526],
       [-0.14194088,  0.71295599, -0.10051528, -0.11628008, -0.26955969,
        -0.15300985,  0.45661472,  0.54125506]]), h=array([[ 0.17016332,  0.06186617,  0.15829436,  0.16037617, -0.15841691,
        -0.3717137 ,  0.11021478,  0.22602701],
       [-0.05230012,  0.42949286, -0.07228138, -0.04229116, -0.11640012,
        -0.06838009,  0.30524024,  0.28048882]]))

In [41]:
# Convert this notebook for Docs
! jupyter nbconvert --to markdown --output-dir ../docs/_posts 2017-11-11-tf.nn.rnn-APIs.ipynb

[NbConvertApp] WARNING | pattern '2017-11-11-tf.nn.rnn-APIs.ipynb' matched no files
This application is used to convert notebook files (*.ipynb) to various other
formats.


Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--debug
    set log level to logging.DEBUG (maximize logging output)
--generate-config
    generate default config file
-y
    Answer yes to any questions instead of prompting.
--execute
    Execute the notebook prior to export.
--allow-errors
    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.
--stdin
    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'
--stdout
    Write notebook o